In [2]:
#######
# 영화 후기 데이터 -> NNLM 모델 적용 
#######

### 데이터 읽기
import pandas as pd
from pandas import DataFrame, Series
df_train = pd.read_csv('100 data/ratings_test.txt', delimiter='\t', keep_default_na=False)
print(df_train['document'][:10])

# Data는 100개만 사용
text = df_train['document'][:100]

0                                                  굳 ㅋ
1                                 GDNTOPCLASSINTHECLUB
2               뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아
3                     지루하지는 않은데 완전 막장임... 돈주고 보기에는....
4    3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??
5                                   음악이 주가 된, 최고의 음악영화
6                                              진정한 쓰레기
7             마치 미국애니에서 튀어나온듯한 창의력없는 로봇디자인부터가,고개를 젖게한다
8    갈수록 개판되가는 중국영화 유치하고 내용없음 폼잡다 끝남 말도안되는 무기에 유치한c...
9       이별의 아픔뒤에 찾아오는 새로운 인연의 기쁨 But, 모든 사람이 그렇지는 않네..
Name: document, dtype: object


In [4]:
### 형태소 분석 및 one-hot encoding
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Okt
twitter_tag = Okt()

# 형태소 분석 방법(조사, 어미 등 제외)
def twitter_tokenizer_part(text):
    lst = []
    for tpl in twitter_tag.pos(text, stem=True):
        if not tpl[1] in ["Josa", "Eomi", "PreEomi", "Punctuation"]:
            lst.append(tpl[0])
    if len(lst) != 0: 
        return lst
    else:
        return [""]
    
# CountVectorizer 객체 형성 -> one-hot encoding
vect = CountVectorizer(tokenizer=twitter_tokenizer_part).fit(text)
print(vect.get_feature_names()[80:90])
print(len(vect.get_feature_names()))

['괜히', '굉장하다', '교훈', '구성', '군', '굳다', '굿굿', '궁금', '귀엽다', '그']
622


In [5]:
# 입력할 후기를 토큰화함
tokenized_words = []
for line in text:
    for tpl in twitter_tag.pos(line, stem=True):
        if not tpl[1] in ["Josa", "Eomi", "PreEomi", "Punctuation"]:
            tokenized_words.append(tpl[0])

print(tokenized_words[10:20])
print(len(tokenized_words))

['점', '짜다', '리', '더', '더욱', '아니다', '지루하다', '않다', '완전', '막장']
981


In [7]:
# n-1, n-2, n-3번째 단어를 통해 n번째 단어를 예측하는 학습 데이터 생성
x_train = []
y_train = []
for i in range(len(tokenized_words)-3):
    x_train.append(vect.transform([tokenized_words[i],tokenized_words[i+1],tokenized_words[i+2]]).toarray())
    y_train.append(vect.transform([tokenized_words[i+3]]).toarray())
    
# 데이터 예시
y_train[0]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [11]:
# 리스트를 array로 변형
import numpy as np
x_train = np.array(x_train).reshape(978, 3, 622)
y_train = np.array(y_train).reshape(978, 622)

In [27]:
### 모델 생성 
from keras.models import Model
from keras.layers import Dense, Input, Flatten
from keras.optimizers import SGD

In [25]:
x = Input([3,len(y_train[0])])
x2 = Dense(2)(x)
x3 = Flatten()(x2)
h = Dense(64, activation='tanh')(x3)
y = Dense(len(y_train[0]), activation='softmax')(h)
model = Model(x,y)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 3, 622)            0         
_________________________________________________________________
dense_5 (Dense)              (None, 3, 2)              1246      
_________________________________________________________________
flatten_2 (Flatten)          (None, 6)                 0         
_________________________________________________________________
dense_6 (Dense)              (None, 64)                448       
_________________________________________________________________
dense_7 (Dense)              (None, 622)               40430     
Total params: 42,124
Trainable params: 42,124
Non-trainable params: 0
_________________________________________________________________


In [30]:
sgd = SGD(lr=0.1) # default = 0.01
model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=['accuracy'])

In [31]:
model.fit(x_train, y_train, epochs=1000, batch_size=100)

Epoch 1/1000
978/978 [==============================] - 1s 722us/step - loss: 6.3270 - acc: 0.0072
Epoch 2/1000
978/978 [==============================] - 0s 287us/step - loss: 6.3237 - acc: 0.0276
Epoch 3/1000
978/978 [==============================] - 0s 294us/step - loss: 6.3204 - acc: 0.0276
Epoch 4/1000
978/978 [==============================] - 0s 317us/step - loss: 6.3171 - acc: 0.0256
Epoch 5/1000
978/978 [==============================] - 0s 301us/step - loss: 6.3137 - acc: 0.0286
Epoch 6/1000
978/978 [==============================] - 0s 307us/step - loss: 6.3103 - acc: 0.0297
Epoch 7/1000
978/978 [==============================] - 0s 311us/step - loss: 6.3069 - acc: 0.0276
Epoch 8/1000
978/978 [==============================] - 0s 300us/step - loss: 6.3033 - acc: 0.0276
Epoch 9/1000
978/978 [==============================] - 0s 327us/step - loss: 6.2997 - acc: 0.0256
Epoch 10/1000
978/978 [==============================] - 0s 395us/step - loss: 6.2959 - acc: 0.0276
Epoch 11/

978/978 [==============================] - 0s 277us/step - loss: 1.3901 - acc: 0.8231
Epoch 965/1000
978/978 [==============================] - 0s 361us/step - loss: 1.3788 - acc: 0.8272
Epoch 966/1000
978/978 [==============================] - 0s 312us/step - loss: 1.3751 - acc: 0.8333
Epoch 967/1000
978/978 [==============================] - 0s 363us/step - loss: 1.3718 - acc: 0.8323
Epoch 968/1000
978/978 [==============================] - 0s 368us/step - loss: 1.3724 - acc: 0.8344
Epoch 969/1000
978/978 [==============================] - 0s 338us/step - loss: 1.3702 - acc: 0.8292
Epoch 970/1000
978/978 [==============================] - 0s 351us/step - loss: 1.3638 - acc: 0.8323
Epoch 971/1000
978/978 [==============================] - 0s 359us/step - loss: 1.3569 - acc: 0.8354
Epoch 972/1000
978/978 [==============================] - 0s 341us/step - loss: 1.3578 - acc: 0.8323
Epoch 973/1000
978/978 [==============================] - 0s 341us/step - loss: 1.3551 - acc: 0.8313
Epoch

In [34]:
pre = model.predict(x_train)

In [35]:
for i in range(30):
    print(tokenized_words[i: i+3], end='===>')
    print(pre[i].argmax(), end=' ')
    print(vect.get_feature_names()[pre[i].argmax()])

['굳다', 'ㅋ', 'GDNTOPCLASSINTHECLUB']===>277 보다
['ㅋ', 'GDNTOPCLASSINTHECLUB', '뭐']===>450 이
['GDNTOPCLASSINTHECLUB', '뭐', '이']===>585 평점
['뭐', '이', '평점']===>185 들다
['이', '평점', '들']===>119 나쁘다
['평점', '들', '나쁘다']===>374 않다
['들', '나쁘다', '않다']===>1 10
['나쁘다', '않다', '10']===>496 점
['않다', '10', '점']===>539 짜다
['10', '점', '짜다']===>55 개막
['점', '짜다', '리']===>165 더
['짜다', '리', '더']===>166 더욱
['리', '더', '더욱']===>356 아니다
['더', '더욱', '아니다']===>528 지루하다
['더욱', '아니다', '지루하다']===>374 않다
['아니다', '지루하다', '않다']===>375 알
['지루하다', '않다', '완전']===>488 재밌다
['않다', '완전', '막장']===>473 임
['완전', '막장', '임']===>172 돈
['막장', '임', '돈']===>474 있다
['임', '돈', '주다']===>276 보기
['돈', '주다', '보기']===>10 3
['주다', '보기', '3']===>20 d
['보기', '3', 'D']===>220 만
['3', 'D', '만']===>296 빠지다
['D', '만', '아니다']===>273 별
['만', '아니다', '별']===>152 다섯
['아니다', '별', '다섯']===>53 개
['별', '다섯', '개']===>515 주다
['다섯', '개', '주다']===>426 왜
